In [1]:
import torch
import os
import numpy as np
from tqdm import tqdm
from sklearn.metrics import roc_curve

In [2]:
from shutil import copy2

In [3]:
import matplotlib.pyplot as plt

In [4]:
races = ['African','Asian','Caucasian','Indian']
labels = []
dist = []
for race in races:
        with open(f"./sims/{race}_sims.csv",'r') as f:
            lines = f.readlines()
        for line in lines:
            arr = line.strip().split(',')
            labels.append(int(arr[3]))
            dist.append(float(arr[2]))

In [5]:
fpr , tpr ,thresholds = roc_curve(labels,dist)
idx = np.argmax(tpr-fpr)
threshold = thresholds[idx]
print(f"Threshold: {threshold}")

Threshold: 0.33063241839408875


In [6]:
races = ['African','Asian','Caucasian','Indian']
accuracies = {}
for race in races:
        accuracies[f"{race}"] = {'TP':0,'TN':0,'FP':0,'FN':0}
        with open(f"./sims/{race}_sims.csv",'r') as f:
            lines = f.readlines()
        dist = []
        labels = []
        for line in lines:
            arr = line.strip().split(',')
            dist.append(float(arr[2]))
            labels.append(int(arr[3]))
            if float(arr[2]) < threshold:
                with open(f"./results/{race}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},0,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}"]['TN'] += 1
                else:
                    accuracies[f"{race}"]['FN'] += 1
            else:
                with open(f"./results/{race}_results.csv",'a') as f:
                    f.write(f"{arr[0]},{arr[1]},1,{arr[3]}\n")
                if int(arr[3]) == 1:
                    accuracies[f"{race}"]['TP'] += 1
                else:
                    accuracies[f"{race}"]['FP'] += 1
        fpr, tpr, thresholds = roc_curve(labels,dist)
        fpr = np.array(fpr)
        tpr = np.array(tpr)
        with open(f"./roc/{race}_roc.npy",'wb') as f:
            np.save(f,fpr)
            np.save(f,tpr)

In [7]:
print(accuracies)

{'African': {'TP': 2877, 'TN': 123, 'FP': 111, 'FN': 2889}, 'Asian': {'TP': 2851, 'TN': 149, 'FP': 98, 'FN': 2902}, 'Caucasian': {'TP': 2901, 'TN': 99, 'FP': 4, 'FN': 2996}, 'Indian': {'TP': 2900, 'TN': 100, 'FP': 74, 'FN': 2925}}


In [10]:
gar = {}
far = {}
scores = {}
for race in races:
        tp = accuracies[f"{race}"]['TP']
        tn = accuracies[f"{race}"]['TN']
        fp = accuracies[f"{race}"]['FP']
        fn = accuracies[f"{race}"]['FN']
        gar[f"{race}"] = tp/(tn+tp)
        far[f"{race}"] = fp/(fn+fp)
        scores[f"{race}"] = 100*(fn+tp)/(fn+fp+tn+tp)

In [11]:
gar

{'African': 0.959,
 'Asian': 0.9503333333333334,
 'Caucasian': 0.967,
 'Indian': 0.9666666666666667}

In [12]:
far

{'African': 0.037,
 'Asian': 0.03266666666666666,
 'Caucasian': 0.0013333333333333333,
 'Indian': 0.024674891630543514}

In [13]:
scores

{'African': 96.1,
 'Asian': 95.88333333333334,
 'Caucasian': 98.28333333333333,
 'Indian': 97.09951658609768}